# Hands-on 8: Circuit Optimizers

Today, most hardware has connectivity and control constraints, ie only some two-qubits gates can only be executed on few pairs of qubits. Pairs of qubits that can interact define the connectivity constraints of a hardware. Solving *the swap insertion problem* consists in inserting SWAP gates in a circuit to make it compliant with connectivity constraints of a specific hardware. The QLM provides a plugin **Nnizer** implementing algorithms to solve *the swap insertion problem*. The **PBO** plugin to replace gates by realistic implementations.

For our example we are going to use the previous maxcut problem used to illustrate QAOA:

In [ ]:
#Import networkx 
import networkx as nx

#Create our simple graph
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3, 4])
G.add_edge(0, 1)
G.add_edge(0, 4)
G.add_edge(1, 2)
G.add_edge(1, 4)
G.add_edge(2, 3)
G.add_edge(3, 4)


#Plot our graph
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
nodes_positions = nx.spring_layout(G, iterations=len(G.nodes())*100)
nx.draw_networkx(G, 
                 pos=nodes_positions, 
                 node_color='#4EEA6A', 
                 node_size=440, 
                 font_size=14)
plt.show()

from qat.vsolve.qaoa import MaxCut
problem = MaxCut(G)
print(problem)
ansatz = problem.qaoa_ansatz(1)

The circuit used to begin with is the following one:

In [ ]:
circuit = ansatz.circuit
%qatdisplay XXX

## NNizer

**Defining hardware specifications**

Connectivity constraints can be defined by the class `qat.core.Topology`. Few topologies are predefined:
 - **ALL_TO_ALL**: There is no connectivity constraints
 - **LNN**: Gates can only be applied between qubits of index $i$ and $i + 1$ ($\forall i \in [\![0; n - 1]\!]$)
 - **CUSTOM**: Custom constraints
 
Let's define our custom topology.

For instance, considering a fake quantum computer which has few connectivity constraints defined by the following graph:

<img src="topology.png" width="200px"/>


On this computer, two qubits gates can then only be executed using one of the following pairs: `[(0, 1), (0, 2), (1, 3), (2, 3), (2, 4), (3, 4)]` (corresponding to the edges of the previous graph).

In [ ]:
#Import Topology from qat.core
from qat.core import XXX

# Define your custom topology using Topology
my_topology = XXX()

#Add the constraints corresponding to the fake quantum computer
for i, j in [(XXX, XXX), (XXX, XXX), (XXX, XXX),
             (XXX, XXX), (XXX, XXX), (XXX, XXX)]:
    my_topology.add_edge(i, j)

To make a circuit compliant with the hardware specifications, one need to manage the hardware specifications (and not only the topology).
The Python class `qat.core.HardwareSpecs` defines specifications of an hardware. For instance, the following code defines specifications of my hardware.

In [ ]:
#Import HardwareSpecs from qat.core
from qat.core import XXX

# Define an hardware: 5 qubits + connectivity constraints using your topology
my_hardware = XXX(nbqbits=XXX, topology=XXX)

**Solving the swap insertion problem using compile method**

The first way of using Nnizer is by using the compile method.

To do so, we need to put our ansatz in a batch so we can feed it to Nnizer.

---
***Why the circuit is wrapped into a Job which is wrapped into a Batch?***

The nnizer is a plugin, which means that the nnizer can extend a QPU. A QPU executes batches which is a set of circuits with execution parameters for each circuit (like the number of samples, the list of measured qubits, ...). Being a plugin means nnizer must receives batches

---

In [ ]:
#import Nnizer from qat.plugins
from qat.plugins import XXX
#import Batch from qat.core
from qat.core import XXX

#Create the nnizer using Nnizer and lets specify atos for the method
nnizer = XXX(method="XXX")

#Use the nnizer's compile method to nnize our circuit named ansatz and specifying the hardware
nnized_batch = nnizer.compile(Batch(jobs=[XXX]),
                              XXX)

#get the circuit to have a look at it and see the inserted SWAP gates
nnized_ansatz_circuit = nnized_batch.jobs[0].circuit
%qatdisplay nnized_ansatz_circuit
print("Number of gates: ", len(nnized_ansatz_circuit.ops))

Use the following cell to print the circuit before using Nnizer and see the number of gates in it:

In [ ]:
%qatdisplay circuit
print("Number of gates: ", len(circuit.ops))

Using Nnizer change the final order of the qubits:

In [ ]:
print(nnized_batch.jobs[0].qubits)

To take this into account for the minimization part, you should use the previous job that's keep this information, namely nnized_batch.jobs[0].


In [ ]:
from qat.qpus import get_default_qpu
from qat.plugins import QuameleonPlugin
from qat.plugins import ScipyMinimizePlugin

qpu = get_default_qpu()
qpu = QuameleonPlugin(specs=my_hardware)| qpu
stack = ScipyMinimizePlugin(method="COBYLA",
                            tol=1e-2, 
                            options={"maxiter":150}) | qpu
# We can directly call the to_job method of the Problem class to pack an Ansatz and 
# the cost observable in a single abstract Job

job=nnized_batch.jobs[0]
result = stack.submit(XXX)
print("Final energy:", result.value)


One way to deal with the last step is simply to add the permutation when specifying the sampling job:

In [ ]:
import numpy as np
#Retrieving the optimized parameters:
params = eval(result.meta_data['parameters'])

#Binding the variables:
sol_job = job(**{key: var for key, var in zip(job.get_variables(), params)})

#Checking that this indeeds gives the optimized energy
sol_res = qpu.submit(sol_job)
print("Check, energy =", sol_res.value)

#Rerunning in 'SAMPLE' mode to get the most probable states:
sampling_job = sol_job.circuit.to_job(qubits=XXX)
sol_res = qpu.submit(sampling_job)
print("Most probable states are:")
for sample in sol_res:
    if sample.probability > 0.05:
        print(sample.state, sample.probability)
# We can also directly cast states into bitstrings for practical use:
print("And as bitstrings:")
for sample in sol_res:
    if sample.probability > 0.05:
        print(sample.state.bitstring,  sample.probability)
        indices_bin_1 = np.where(np.array(list(sample.state.bitstring), dtype=int) == 1)[0]
        indices_bin_0 = np.where(np.array(list(sample.state.bitstring), dtype=int) == 0)[0]
        print("0 list : "+ str(indices_bin_0))
        print("1 list : " + str(indices_bin_1) + "\n")
        
        plt.figure(figsize=(8, 8))
        node_size = 440
        font_size = 14
        nx.draw_networkx(G, 
                         pos=nodes_positions, 
                         nodelist=indices_bin_1.tolist(), 
                         node_color='#FFE033', 
                         node_size=node_size, 
                         font_size=font_size)

        nx.draw_networkx(G, 
                         pos=nodes_positions, 
                         nodelist=indices_bin_0.tolist(), 
                         node_color='#7B9BF2', 
                         node_size=node_size, 
                         font_size=font_size)

        nx.draw_networkx_edges(G, pos=nodes_positions)
        plt.show()


**Solving the swap insertion problem by creating a stack**

Another way of dealing with the swap insertion problem is by using a stack with Nnizer in it.

First we will use QuameleonPlugin that will integrate the constraints of our targeted hardware to our stack.

Once QuameleonPlugin in the stack operations that are not compliant with the constraint of the hardware will not be allowed:

In [ ]:
from qat.qpus import LinAlg
#Import QuameleonPlugin from qat.plugins
from qat.plugins import XXX

#Create a qpu with QuameleonPlugin using my_hardware as a parameter (defined previously) and LinAlg as the simulator
qpu = QuameleonPlugin(specs=XXX)| XXX()

#Now create a stack using the qpu you have just defined
#And add to it ScipyMinimizePlugin to solve the optimization problem
stack =  XXX(method="COBYLA",
                            tol=1e-2, 
                            options={"maxiter":150})| XXX

from qat.comm.exceptions.ttypes import QPUException
try:
    #Submit your ansatz to your stack
    result = XXX.submit(XXX)

except QPUException as exception:
    print("The job (not processed by the Nnizer) can't be " +
          "executed by the QPU:\n" + exception.message)

As you can see it is not possible to execute our circuit due to the topology.

To simply solve this issue, you can add to the stack the plugin Nnizer.

In [ ]:
from qat.qpus import LinAlg
from qat.plugins import ScipyMinimizePlugin
from qat.plugins import QuameleonPlugin
from qat.plugins import XXX

qpu = QuameleonPlugin(specs=my_hardware)| LinAlg()
stack = XXX(method="atos") | ScipyMinimizePlugin(method="COBYLA",
                            tol=1e-2, 
                            options={"maxiter":150})| qpu

result = stack.submit(ansatz)
print("Final energy:", result.value)

As you can see the circuit can now be solve due to Nnizer making the circuit compliant with the constraints.

Few methods are implemented in **Nnize** to solve the swap insertion problem:
 - **atos**: based on a strict generalization of the algorithm described in [An Efficient Method to Convert Arbitrary Quantum Circuits to Ones on a Linear Nearest Neighbor Architecture](https://ieeexplore.ieee.org/document/4782917) by *Hirata and al.*
 - **sabre**: implementation of [Tackling the Qubit Mapping Problem for NISQ-Era Quantum Devices](https://dl.acm.org/citation.cfm?id=3304023) by *Gushu Li, Yufei Ding and Yuan Xie*
 - **bka**: implementation of [Efficient mapping of quantum circuits to the IBM QX architectures](https://ieeexplore.ieee.org/document/8342181) by *Alwin Zulehner, Alexandru Paler and Robert Wille*
 - **pbn**: based on a strict generalization of the algorithm described in [Synthesis of quantum circuits for linear nearest neighbor architectures](https://link.springer.com/article/10.1007/s11128-010-0201-2) by *Mehdi Saeedi, Robert Wille and Rolf Drechsler*
 
We have used the **atos** algorithm to solve the problem.

To get the results as previously we just have to define a stack composed of Nnizer and the qpu so we can submit the last run with the best parameters obtained during the optimization phase.

In [ ]:
#Create a stack without ScipyMinimizePlugin
stack = XXX(method="atos")| qpu

#Retrieving the optimized parameters:
params = eval(result.meta_data['parameters'])

#Binding the variables:
sol_job = ansatz(**{key: var for key, var in zip(ansatz.get_variables(), params)})

#Checking that this indeeds gives the optimized energy
sol_res = stack.submit(sol_job)
print("Check, energy =", sol_res.value)

#Rerunning in 'SAMPLE' mode to get the most probable states:
sampling_job = sol_job.circuit.to_job()
sol_res = stack.submit(sampling_job)
print("Most probable states are:")
for sample in sol_res:
    if sample.probability > 0.05:
        print(sample.state, sample.probability)
# We can also directly cast states into bitstrings for practical use:
print("And as bitstrings:")
for sample in sol_res:
    if sample.probability > 0.05:
        print(sample.state.bitstring,  sample.probability)
        indices_bin_1 = np.where(np.array(list(sample.state.bitstring), dtype=int) == 1)[0]
        indices_bin_0 = np.where(np.array(list(sample.state.bitstring), dtype=int) == 0)[0]
        print("0 list : "+ str(indices_bin_0))
        print("1 list : " + str(indices_bin_1) + "\n")
        
        plt.figure(figsize=(8, 8))
        node_size = 440
        font_size = 14
        nx.draw_networkx(G, 
                         pos=nodes_positions, 
                         nodelist=indices_bin_1.tolist(), 
                         node_color='#FFE033', 
                         node_size=node_size, 
                         font_size=font_size)

        nx.draw_networkx(G, 
                         pos=nodes_positions, 
                         nodelist=indices_bin_0.tolist(), 
                         node_color='#7B9BF2', 
                         node_size=node_size, 
                         font_size=font_size)

        nx.draw_networkx_edges(G, pos=nodes_positions)
        plt.show()


## PBO
Now what we want to take into consideration is the real implementation of the SWAP gates:
<img src="swap.png" />
For example the following equivalent circuit:
<img src="equi.png"/>
To do so we will use the previous nnized ansatz:

In [ ]:
%qatdisplay nnized_ansatz_circuit

We are going to need a class named GraphCircuit in qat.pbo to create a graph from our circuit:

In [ ]:
from qat.pbo import XXX

# Generate a graph
graph = XXX()

# Transform the circuit into a graph
graph.load_circuit(XXX)

Once that is done we can create a pattern to replace:
+ old_pattern will be the SWAP operation
+ new_pattern will be the equivalent pattern of CNOTs

In [ ]:
# Pattern
old_pattern = [('XXX', [XXX, 1])]
new_pattern = [('XXX', [XXX, XXX]), ('XXX', [XXX, XXX]), ('XXX', [XXX, XXX])]


Calling replace_pattern on our graph with the old and the new pattern as argument will replace the first pattern in the graph.

In [ ]:
# Replace pattern
graph.XXX(XXX, XXX)

To replace all the patterns in the graph it is enough to use a while loop with the previous call:

In [ ]:
# Replace all pattern
XXX graph.replace_pattern(old_pattern, new_pattern):
    continue

In the end you can display the circuit with CNOTs instead of SWAP gates:

In [ ]:
circ_no_swap = graph.to_circ()
%qatdisplay circ_no_swap